In [1]:
using Pkg
Pkg.add("Gridap")
Pkg.add("Plots")
using Gridap
using Gridap.Geometry  # For Point in plotting
using Plots

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
   2239.4 ms  ? GridapODEs
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
   1885.5 ms  ? GridapODEs


In [2]:
rho_p = 2500.0  # Particle density (kg/m³)
rho_f = 1000.0  # Fluid density (kg/m³)
g = 9.81  # Gravity (m/s²)
dp = 1.0  # Drag coefficient (simplified constant, units 1/s)
Cvm = 0.5  # Virtual mass coefficient
L = 10.0  # Wellbore length (m)
n = 100  # Number of cells
dt = 0.01  # Time step (s)
t_end = 1.0  # Final time (s)
theta_s(x) = 0.0  # Inclination (horizontal)
bp(phi) = 0.0  # Dispersive pressure (simplified zero)

bp (generic function with 1 method)

In [3]:
domain = (0.0, L)  # 1D domain
partition = (n,)  # 1D partition
model = CartesianDiscreteModel(domain, partition; isperiodic=(true,))

CartesianDiscreteModel()

In [4]:
order = 1
reffe_scalar = ReferenceFE(lagrangian, Float64, order)

(Lagrangian(), (Float64, 1), Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}())

In [5]:
V_phi = TestFESpace(model, reffe_scalar; conformity=:H1)
V_vp = TestFESpace(model, reffe_scalar; conformity=:H1)
V_vf = TestFESpace(model, reffe_scalar; conformity=:H1)
Q_lambda = TestFESpace(model, reffe_scalar; conformity=:H1)  # H1 for λ (periodic)
Y = MultiFieldFESpace([V_phi, V_vp, V_vf, Q_lambda])

MultiFieldFESpace()

In [6]:
U_phi = TransientTrialFESpace(V_phi)
U_vp = TransientTrialFESpace(V_vp)
U_vf = TransientTrialFESpace(V_vf)
U_lambda = TransientTrialFESpace(Q_lambda)
X = TransientMultiFieldFESpace([U_phi, U_vp, U_vf, U_lambda])

MultiFieldFESpace()

In [7]:
degree = 2 * order
Omega = Triangulation(model)
dOmega = Measure(Omega, degree)

GenericMeasure()

In [8]:
phi_f(phi) = 1.0 - phi  # Fluid fraction
sin_theta = theta_s  # For gravity (zero here)

theta_s (generic function with 1 method)

In [9]:
function res(t, u, v)
  dtu = ∂t(u)
  phi, vp, vf, lam = u
  dphi, dvp, dvf, dlam = dtu
  test_phi, test_chi, test_wp, test_wf = v
  phiF = phi_f(phi)
  ∫( test_phi * dphi - ∇(test_phi) ⋅ (phi * vp) )dOmega +
  ∫( test_chi * (-dphi) - ∇(test_chi) ⋅ (phiF * vf) )dOmega +
  ∫( test_wp * rho_p * (dvp + vp * ∇(vp)) - test_wp * rho_p * g * sin_theta - test_wp * dp * (vf - vp) -
     test_wp * Cvm * rho_f * phi * (dvf + vf * ∇(vf) - dvp - vp * ∇(vp)) +
     ∇(test_wp) ⋅ phi * lam + ∇(test_wp) ⋅ bp(phi) )dOmega +
  ∫( test_wf * rho_f * (dvf + vf * ∇(vf)) - test_wf * rho_f * g * sin_theta + test_wf * dp * (vf - vp) +
     test_wf * Cvm * rho_f * phi * (dvf + vf * ∇(vf) - dvp - vp * ∇(vp)) +
     ∇(test_wf) ⋅ phiF * lam )dOmega
end

res (generic function with 1 method)

In [10]:
function jac(t, u, du, v)
  dtu = ∂t(u)
  phi, vp, vf, lam = u
  dphi_u, dvp_u, dvf_u, dlam_u = dtu  # Time derivs from u
  dphi_y, dvp_y, dvf_y, dlam_y = du
  test_phi, test_chi, test_wp, test_wf = v
  phiF = phi_f(phi)
  dphiF = -dphi_y
  ∫( - ∇(test_phi) ⋅ (dphi_y * vp + phi * dvp_y) )dOmega +
  ∫( - ∇(test_chi) ⋅ (dphiF * vf + phiF * dvf_y) )dOmega +
  ∫( test_wp * rho_p * (vp * ∇(dvp_y) + dvp_y * ∇(vp)) - test_wp * dp * (dvf_y - dvp_y) -
     test_wp * Cvm * rho_f * (dphi_y * (dvf_u + vf * ∇(vf) - dvp_u - vp * ∇(vp)) + phi * (vf * ∇(dvf_y) + dvf_y * ∇(vf) - vp * ∇(dvp_y) - dvp_y * ∇(vp))) +
     ∇(test_wp) ⋅ dphi_y * lam )dOmega +
  ∫( test_wf * rho_f * (vf * ∇(dvf_y) + dvf_y * ∇(vf)) + test_wf * dp * (dvf_y - dvp_y) +
     test_wf * Cvm * rho_f * (dphi_y * (dvf_u + vf * ∇(vf) - dvp_u - vp * ∇(vp)) + phi * (vf * ∇(dvf_y) + dvf_y * ∇(vf) - vp * ∇(dvp_y) - dvp_y * ∇(vp))) +
     ∇(test_wf) ⋅ dphiF * lam )dOmega +
  ∫( ∇(test_wp) ⋅ phi * dlam_y + ∇(test_wf) ⋅ phiF * dlam_y )dOmega
end

function jac_t(t, u, dut, v)
  phi, vp, vf, lam = u
  ddphi, ddvp, ddvf, ddlam = dut
  test_phi, test_chi, test_wp, test_wf = v
  ∫( test_phi * ddphi )dOmega +
  ∫( test_chi * (-ddphi) )dOmega +
  ∫( test_wp * rho_p * ddvp - test_wp * Cvm * rho_f * phi * (ddvf - ddvp) )dOmega +
  ∫( test_wf * rho_f * ddvf + test_wf * Cvm * rho_f * phi * (ddvf - ddvp) )dOmega
end

jac_t (generic function with 1 method)

In [11]:
op = TransientFEOperator(res, jac, jac_t, X, Y)
ls = LUSolver()
nls = NLSolver(ls; method=:newton, iterations=10, show_trace=true)
theta = 0.5
solver = ThetaMethod(nls, dt, theta)
# Initial conditions (constant for simplicity)
phi0(x) = 0.1
vp0(x) = 1.0
vf0(x) = 1.0
lam0(x) = 0.0
y0 = interpolate_everywhere([phi0, vp0, vf0, lam0], X(0.0))
sol_t = solve(solver, op, y0, 0.0, t_end)

Gridap.ODEs.TransientFETools.TransientFESolution(GenericODESolution(), MultiFieldFESpace())

In [26]:
n_points = 100
s_vals = range(0.0, L, n_points)
points = [Point(s) for s in s_vals] # 1D points
# Advance to final time and plot
for (t, yh) in sol_t
  if t ≈ t_end
    phi_h, vp_h, vf_h, lam_h = yh
    phi_vals = [phi_h(pt) for pt in points]
    vp_vals = [vp_h(pt) for pt in points]
    vf_vals = [vf_h(pt) for pt in points]
    p1 = plot(s_vals, phi_vals, label="ϕ_p", xlabel="s (m)", ylabel="ϕ_p", title="Particle Fraction at t=$t")
    p2 = plot(s_vals, vp_vals, label="v_p", xlabel="s (m)", ylabel="Velocity (m/s)", title="Velocities at t=$t")
    plot!(p2, s_vals, vf_vals, label="v_f")
    display(plot(p1, p2, layout=(2,1)))
  end
end

LoadError: MethodError: no method matching Float64(::VectorValue{1, Float64})
The type `Float64` exists, but no method is defined for this combination of argument types when trying to construct it.

[0mClosest candidates are:
[0m  (::Type{T})(::T) where T<:Number
[0m[90m   @[39m [90mCore[39m [90m[4mboot.jl:900[24m[39m
[0m  Float64([91m::IrrationalConstants.Quartπ[39m)
[0m[90m   @[39m [36mIrrationalConstants[39m [90m~/.julia/packages/IrrationalConstants/lWTip/src/[39m[90m[4mmacro.jl:131[24m[39m
[0m  Float64([91m::UInt8[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:245[24m[39m
[0m  ...
